In [47]:

#Parameters
GROUP_ID = 'Group36'
ALGORITHM = 'ga'#'bt' - Backtracking | 'fc' - Foward Checking | 'ac3' arc-consistency, 'ga' - Genetic Algo | 'sa' Simulated Annealing
PUZZLE_TYPE = 'NA' #Leave empty
PUZZLE_PATH = 'Puzzles/Easy-P2.txt' #Insert file path for desired puzzle

In [48]:
#HELPER FUNCTIONS
def read_puzzle(file_path):
    #read puzzle from file
    puzzle = []
    with open(file_path, 'r') as f:
        for line in f:
            row = []
            for cell in line.strip().split(','):
                c = cell.strip()
                if c == '' or c== '?' or not c.isdigit():
                    row.append('?')
                else:
                    row.append(c)
            puzzle.append(row)  
    return puzzle

def write_puzzle(puzzle, group_id, algorithm, puzzle_type, puzzle_number):
    #write puzzle to file
    filename = f"{group_id}_{algorithm}_{puzzle_type}_{puzzle_number}.txt"
    with open(filename, 'w') as f:
        for row in puzzle:
            f.write(','.join(str(cell) for cell in row) + '\n')
    print(f"Puzzle written to {filename}")

def get_puzzle_number(puzzle_path):
    #get puzzle number from path
    b = puzzle_path.split('/')[-1]
    return b.split('.')[0]

def is_valid_sudoku(puzzle):

	def valid_group(group):
		return sorted(group) == [str(i) for i in range(1, 10)]

	#check rows
	for row in puzzle:
		if not valid_group(row):
			return False

	#check cols
	for col in range(9):
		if not valid_group([puzzle[row][col] for row in range(9)]):
			return False

	#check boxes
	for box_row in range(0, 9, 3):
		for box_col in range(0, 9, 3):
			box = []
			for i in range(3):
				for j in range(3):
					box.append(puzzle[box_row + i][box_col + j])
			if not valid_group(box):
				return False
	return True

In [49]:
#BACKTRACKING SOLVER

def find_empty(puzzle):
    #find empty
    for i in range(9):
        for j in range(9):
            if puzzle[i][j] == '?' or puzzle[i][j] == '0':
                return (i, j)
    return None

def is_valid(puzzle, row, col, num):
    #check row
    num = str(num)
    if num in puzzle[row]:
        return False
    for i in range(9):
    #check col
        if puzzle[i][col] == num:
            return False

    start_row, start_col = 3 * (row // 3), 3 * (col // 3)
    #check box
    for i in range(3):
        for j in range(3):
            if puzzle[start_row + i][start_col + j] == num:
                return False
    return True

def solve_sudoku_bt(puzzle, assignments=None):
    if assignments is None:
        assignments = {'count':0}
    #start
    empty = find_empty(puzzle)
    #done if no empty
    if not empty:
        return True, assignments['count']
    row, col = empty
    #try 1-9
    for num in range (1, 10):
        if is_valid(puzzle, row, col, num):
            #assign
            puzzle[row][col] = str(num)
            assignments['count'] += 1
            solved, _ = solve_sudoku_bt(puzzle, assignments)
            if solved:
                #solved
                return True, assignments['count']
            #reset
            puzzle[row][col] = '?'
    return False, assignments['count']

if (ALGORITHM == "bt"):
    PUZZLE = read_puzzle(PUZZLE_PATH)
    PUZZLE_NUMBER = get_puzzle_number(PUZZLE_PATH)
    
    solved, assignments = solve_sudoku_bt(PUZZLE)
    if solved:
        write_puzzle(PUZZLE, GROUP_ID, ALGORITHM, PUZZLE_TYPE, PUZZLE_NUMBER)
        print(f"Solved with backtracking! Used {assignments} assignments.")
        if is_valid_sudoku(PUZZLE):
            print("Solution is valid")
        else:
            print("Solution is invalid")
    else:
        print("No solution found by backtracking.")



In [50]:
#FORWARD CHECKING SOLVER

def solve_sudoku_fc(puzzle, assignments = None):
    if assignments is None:
        assignments = {'count':0}
    DIGITS = set('123456789')

    def neighbors_of(row, col):     #computes the peer coordinates
        peers = set()
        for i in range(9):
            if i != row:
                peers.add((i, col))
        for j in range(9):
            if j != col:
                peers.add((row, j))
        start_row, start_col = 3 * (row // 3), 3 * (col // 3)
        for i in range(3):
            for j in range(3):
                rrow, rcol = start_row + i, start_col + j
                if (i, j) != (row % 3, col % 3):
                    peers.add((rrow, rcol))
        return peers

    def initial_domains():      #domain is possible DIGITS without its peer values
        domains = {}
        for row_i in range(9):
            for col_i in range(9):
                cell_value = puzzle[row_i][col_i]
                if cell_value in DIGITS:
                    domains[(row_i, col_i)] = {cell_value}
                    continue
                peer_values = { puzzle[peer_row][peer_col] for peer_row, peer_col in neighbors_of(row_i, col_i)
                                if puzzle[peer_row][peer_col] in DIGITS }

                available_values = DIGITS - peer_values
                domains[(row_i, col_i)] = available_values
        return domains

    def find_empty_cell(grid):
        for row_i in range(9):
            for col_i in range(9):
                if grid[row_i][col_i] == '?' or grid[row_i][col_i] == '0':
                    return row_i, col_i
        return None

    #stats for pruning, assignments, and empty domains
    statistics = {
        'candidates_pruned': 0,
        'assignments': 0,
        'immediately_false': 0
    }

    domains = initial_domains()

    for domain in domains.values():
        if len(domain) == 0:
            statistics['immediately_false'] += 1
            return False, assignments['count']

    worklist = [        #create a list for cells with only 1 possible value
        (r,c) for (r,c), domain in domains.items()
        if (puzzle[r][c] not in DIGITS) and len(domain) == 1
    ]

    while worklist:     #assign only possible value and then prune from peers
        row, col = worklist.pop()
        if puzzle[row][col] in DIGITS:
            continue

        value = next(iter(domains[(row,col)]))
        puzzle[row][col] = value
        domains[(row,col)] = {value}
        assignments['count'] += 1
        statistics['assignments'] += 1

        for peer_row, peer_col in neighbors_of(row, col):       #check forward for peer domains
            if puzzle[peer_row][peer_col] not in DIGITS:
                peer_domain = domains[(peer_row,peer_col)]
                if value in peer_domain:
                    peer_domain.remove(value)
                    statistics['candidates_pruned'] += 1

                    if len(peer_domain) == 0:
                        statistics['immediately_false'] += 1
                        return False, assignments['count']

                    if len(peer_domain) == 1:
                        worklist.append((peer_row,peer_col))

    if find_empty_cell(puzzle) is None:         #complete if grid is filled
        return True, assignments['count']

    #apply backtracking to complete the search after applying forward checking
    applied_backtracking, final_assignments = solve_sudoku_bt(puzzle, assignments)
    return applied_backtracking, final_assignments      #return statistics for total pruned

if (ALGORITHM == "fc"):
    PUZZLE = read_puzzle(PUZZLE_PATH)
    PUZZLE_NUMBER = get_puzzle_number(PUZZLE_PATH)
    
    solved, assignments = solve_sudoku_fc(PUZZLE)
    if solved:
        write_puzzle(PUZZLE, GROUP_ID, ALGORITHM, PUZZLE_TYPE, PUZZLE_NUMBER)
        print(f"Solved with forward checking! Used {assignments} assignments.")
        if is_valid_sudoku(PUZZLE):
            print("Solution is valid")
        else:
            print("Solution is invalid")
    else:
        print("No solution found by forward checking.")

In [51]:
#AC-3 SOLVER
def solve_sudoku_ac3(puzzle):
    #get neighbors for each cell
    def get_neighbors():
        neighbors = {}
        for i in range(9):
            for j in range(9):
                cell = (i, j)
                units = [(i, k) for k in range(9) if k != j]  # same row
                units += [(k, j) for k in range(9) if k != i]  # same col
                box_row, box_col = 3 * (i // 3), 3 * (j // 3)
                for r in range(box_row, box_row + 3):
                    for c in range(box_col, box_col + 3):
                        if (r, c) != cell and (r, c) not in units:
                            units.append((r, c))  # same box
                neighbors[cell] = set(units)
        return neighbors

    #setup domains for each cell
    def init_domains(puzzle):
        domains = {}
        for i in range(9):
            for j in range(9):
                if puzzle[i][j] == '?' or puzzle[i][j] == '0':
                    domains[(i, j)] = set(str(d) for d in range(1, 10))  # all options
                else:
                    domains[(i, j)] = set([puzzle[i][j]])  # already filled
        return domains

    #try to make puzzle easier before backtracking
    def ac3(puzzle):
        assignments = 0
        neighbors = get_neighbors()
        domains = init_domains(puzzle)
        queue = [(xi, xj) for xi in domains for xj in neighbors[xi]]


    #remove impossible values
        def revise(xi, xj):
            revised = False
            to_remove = set()
            for x in domains[xi]:
                if not any(x != y for y in domains[xj]):
                    to_remove.add(x)
            if to_remove:
                domains[xi] -= to_remove
                revised = True
            return revised

    #revise all arcs
        while queue:
            xi, xj = queue.pop(0)
            if revise(xi, xj):
                if not domains[xi]:
                    return False, assignments
                for xk in neighbors[xi]:
                    if xk != xj:
                        queue.append((xk, xi))

    #fill cells with only one option
        for (i, j), vals in domains.items():
            if len(vals) == 1:
                puzzle[i][j] = next(iter(vals))
                assignments += 1
        return True, assignments

    #run ac3 first
    ac3_solved, ac3_assignments = ac3(puzzle)
    #if ac3 didn't finish use backtracking
    if not ac3_solved or any(cell == '?' or cell == '0' for row in puzzle for cell in row):
        bt_solved, bt_assignments = solve_sudoku_bt(puzzle)
        total_assignments = ac3_assignments + bt_assignments
        return bt_solved, total_assignments
    else:
        return ac3_solved, ac3_assignments

if (ALGORITHM == "ac3"):
    PUZZLE = read_puzzle(PUZZLE_PATH)
    PUZZLE_NUMBER = get_puzzle_number(PUZZLE_PATH)
    
    solved, assignments = solve_sudoku_ac3(PUZZLE)
    if solved:
        write_puzzle(PUZZLE, GROUP_ID, ALGORITHM, PUZZLE_TYPE, PUZZLE_NUMBER)
        print(f"Solved with AC-3! Used {assignments} assignemnts")
        if is_valid_sudoku(PUZZLE):
            print("Solution is valid")
        else:
            print("Solution is invalid")
    else:
        print("No solution found by AC-3.")


In [52]:
#SIMULATED ANNEALING SOLVER
import random
import math

def build_fixed_mask(puzzle):      # if false can be modified
    return [[cell in '123456789' for cell in row] for row in puzzle]


def box_cells(top_row, left_col):   # 3x3 boxes
    return [(top_row + i, left_col + j) for i in range(3) for j in range(3)]


def initial_fill_by_box(puzzle, fixed_mask):    #fill missing digits in each box
    grid = [row[:] for row in puzzle]
    for top_row in (0, 3, 6):
        for left_col in (0, 3, 6):
            cells = box_cells(top_row, left_col)
            present = {grid[r][c] for (r, c) in cells if fixed_mask[r][c]}
            missing = [d for d in '123456789' if d not in present]
            random.shuffle(missing)     #randomized start
            k = 0
            for (r, c) in cells:
                if not fixed_mask[r][c]:      #avoid given cells
                    grid[r][c] = missing[k]
                    k += 1
    return grid


def count_conflicts(arr):       #return number of duplicates
    counts = {}
    for item in arr:
        counts[item] = counts.get(item, 0) + 1
    duplicates = sum(count - 1 for count in counts.values() if count > 1)
    return duplicates


def total_energy(grid):     #energy is row & col conflicts, solved at 0
    energy = 0
    for r in range(9):
        energy += count_conflicts(grid[r])
    for c in range(9):
        col_vals = [grid[r][c] for r in range(9)]
        energy += count_conflicts(col_vals)
    return energy


def accept_move(delta_energy, temperature):     #more likely to accept higher E at higher Temp
    if delta_energy < 0:
        return True
    if temperature == 0:
        return False
    return random.random() < math.exp(-delta_energy / temperature)


# solve sudoku with simulated annealing, tracks energy across restarts
def solve_sudoku_sa(puzzle, max_steps=500000, start_temp=1.4, cooling_rate=0.9998, restarts=11):
    assignments = {'count': 0}
    stats = {
        'moves_attempted': 0,
        'moves_accepted': 0,
        'best_energy_per_restart': [],
        'start_energy_per_restart': []
    }
    fixed_mask = build_fixed_mask(puzzle)
    best_solution_grid = None       #track best grid and energy across restarts
    min_energy_overall = float('inf')

    for restart in range(restarts):
        grid = initial_fill_by_box(puzzle, fixed_mask)
        energy = total_energy(grid)
        stats['start_energy_per_restart'].append(energy)
        best_energy_this_restart = energy

        if energy < min_energy_overall:     #update as new best grid
            min_energy_overall = energy
            best_solution_grid = [row[:] for row in grid]

        if energy == 0:     #solved if E = 0
            for r in range(9): puzzle[r] = grid[r][:]
            stats['best_energy_per_restart'].append(0)
            return True, assignments['count'], stats

        temp = start_temp
        for step in range(max_steps // restarts):       #split max steps across restarts
            assignments['count'] += 1
            stats['moves_attempted'] += 1

            box_r, box_c = 3 * random.randint(0, 2), 3 * random.randint(0, 2)
            swappable = [(r, c) for (r, c) in box_cells(box_r, box_c) if not fixed_mask[r][c]]

            if len(swappable) < 2:
                continue

            (r1, c1), (r2, c2) = random.sample(swappable, 2)

            # Calculate local energy change from swap
            old_conflicts = count_conflicts(grid[r1]) + count_conflicts(grid[r2]) + \
                            count_conflicts([grid[i][c1] for i in range(9)]) + \
                            count_conflicts([grid[i][c2] for i in range(9)])

            grid[r1][c1], grid[r2][c2] = grid[r2][c2], grid[r1][c1]     #attempted swap

            new_conflicts = count_conflicts(grid[r1]) + count_conflicts(grid[r2]) + \
                            count_conflicts([grid[i][c1] for i in range(9)]) + \
                            count_conflicts([grid[i][c2] for i in range(9)])

            delta = new_conflicts - old_conflicts       #higher energy is worse

            if accept_move(delta, temp):
                energy += delta
                stats['moves_accepted'] += 1
            else:
                grid[r1][c1], grid[r2][c2] = grid[r2][c2], grid[r1][c1]  # Revert

            if energy < best_energy_this_restart:
                best_energy_this_restart = energy
                if energy < min_energy_overall:
                    min_energy_overall = energy
                    best_solution_grid = [row[:] for row in grid]

            if energy == 0:
                for r in range(9): puzzle[r] = grid[r][:]
                stats['best_energy_per_restart'].append(0)
                return True, assignments['count'] #, stats

            temp *= cooling_rate

        stats['best_energy_per_restart'].append(best_energy_this_restart)

    # if unsolved use the best grid as the puzzle
    if best_solution_grid:
        for r in range(9): puzzle[r] = best_solution_grid[r][:]
                                                        # return stats to view algorithm progress
    return min_energy_overall == 0, assignments['count'] #, stats

if (ALGORITHM == "sa"):
    PUZZLE = read_puzzle(PUZZLE_PATH)
    PUZZLE_NUMBER = get_puzzle_number(PUZZLE_PATH)
    
    solved = solve_sudoku_sa(PUZZLE)
    if solved:
        write_puzzle(PUZZLE, GROUP_ID, ALGORITHM, PUZZLE_TYPE, PUZZLE_NUMBER)
        if is_valid_sudoku(PUZZLE):
            print("Solution is valid")
        else:
            print("Solution is invalid")
    else:
        print("No solution found by Simulated Annealing.")

In [53]:
#GENETIC ALGORITHM SOLVER
import random

def generate_population(puzzle, pop_count):
    population = []
    
    for pop_iteration in range(pop_count):
        temp_puzzle = [row.copy() for row in puzzle]
        
        
        for i in range(9):
            #iteration through rows
            for j in range(9):
            #iteration through cells in the row
                if temp_puzzle[i][j] == "?":
                    rand = str(random.randint(1,9))
                    while rand in temp_puzzle[i]:
                        rand = str(random.randint(1,9))
                    temp_puzzle[i][j] = rand
                    

        population.append(temp_puzzle)

    return population

def count_penalties(puzzle):
    penalty = 0

    for i in range(9):
        row = puzzle[i]
        seen = set()
        for num in row:
            if num != "?" and num in seen:
                penalty += 1
            seen.add(num)

    for j in range(9):
        seen = set()
        for i in range(9):
            num = puzzle[i][j]
            if num != "?" and num in seen:
                penalty += 1
            seen.add(num)

    for box_row in range(0,9,3):
        for box_col in range(0,9,3):
            seen = set()
            for i in range(3):
                for j in range(3):
                    num = puzzle[box_row + i][box_col + j]
                    if num != "?" and num in seen:
                        penalty += 1
                    seen.add(num)

    return penalty


def tournament(population, penalty_counts, tournament_size):
    selected_parents = []


    for _ in range(len(population)):
        #select 3 random indices from population
        tournament_indices = random.sample(range(len(population)), tournament_size)
        #get there fitness scores
        tournament_fitness = [penalty_counts[i] for i in tournament_indices]
        #find min fitness score's index
        winner_index = tournament_indices[tournament_fitness.index(min(tournament_fitness))]
        selected_parents.append(population[winner_index])


    return selected_parents

def repair_puzzle(new, old):

    repaired = [row.copy() for row in new]

    for i in range(9):
        for j in range(9):
            if old[i][j] != "?":
                repaired[i][j] = old[i][j]

    return repaired

def crossover(puzzle, puzzle1, puzzle2):
    cross1 = [row.copy() for row in puzzle1]
    cross2 = [row.copy() for row in puzzle2]
    
    for _ in range(3):  
        crossover_point_row = random.randint(0, 8)
        crossover_point_col = random.randint(0, 8)
        
        for i in range(crossover_point_row, 9):
            for j in range(crossover_point_col, 9):
                if puzzle[i][j] == "?":
                    cross1[i][j], cross2[i][j] = cross2[i][j], cross1[i][j]
    
    return repair_puzzle(cross1, puzzle), repair_puzzle(cross2, puzzle)
    
def mutate(puzzle, puzzle1 , mutation_rate):

    mutated = [row.copy() for row in puzzle1]

    mutable_cells = [(i, j) for i in range(9) for j in range(9) if puzzle[i][j] == "?"]
    
    num_to_mutate = max(1, int(len(mutable_cells) * mutation_rate))
    
    cells_to_mutate = random.sample(mutable_cells, min(num_to_mutate, len(mutable_cells)))
    
    for i, j in cells_to_mutate:
        current_val = mutated[i][j]
        
        # Quick validity check and find alternatives
        alternatives = []
        for num in range(1, 10):
            test_num = str(num)
            if test_num == current_val:
                continue
                
            valid = True
            # Check row
            for col in range(9):
                if col != j and mutated[i][col] == test_num:
                    valid = False
                    break
            if not valid:
                continue
                
            # Check column
            for row in range(9):
                if row != i and mutated[row][j] == test_num:
                    valid = False
                    break
            if not valid:
                continue
                
            # Check box
            start_i, start_j = 3 * (i // 3), 3 * (j // 3)
            for x in range(3):
                for y in range(3):
                    r, c = start_i + x, start_j + y
                    if (r != i or c != j) and mutated[r][c] == test_num:
                        valid = False
                        break
                if not valid:
                    break
            
            if valid:
                alternatives.append(test_num)
        
        if alternatives:
            mutated[i][j] = random.choice(alternatives)
    
    return mutated

def main(puzzle, population, max_generation, population_count, current_generation=0):
    current_generation = 0
    mutate_rate = .3
    tournament_size = 4
    
    while current_generation < max_generation:
        current_generation += 1
        penalty_counts = []
    
        #print(f"generation: {current_generation}")

        best_index = -1
        best_penalty = float('inf')
        best_puzzle = None

        for i in range(population_count):
            penalties = count_penalties(population[i])
            penalty_counts.append(penalties)
        
            if penalties < best_penalty:
                best_penalty = penalties
                best_index = i
                best_puzzle = [row.copy() for row in population[i]]  
                
            if penalties == 0:
                #print(f"I FOUND IT, PUZZLE {i+1}, generation {current_generation}")
                return population[i]

        # Restart population every 30 generations 
        if current_generation % 30 == 0 and current_generation > 0:
            #print("Restarting population due to long-term stagnation!")
            new_pop = [best_puzzle]  
            new_pop.extend(generate_population(puzzle, population_count - 1)) 
            population = new_pop
            continue 

        parents = tournament(population, penalty_counts, tournament_size)

        crossover_parents = []
        for j in range(population_count):
            rand = random.randint(1, population_count-1)
            crossover_parents.extend(crossover(puzzle, parents[j], parents[rand]))
        
        mutate_parents = []
        for k in range(population_count):
            mutate_parents.append(mutate(puzzle, crossover_parents[k], mutate_rate))
    
        population = mutate_parents

    print(f"Max generations ({max_generation}) reached. Best solution had {best_penalty} penalties")
    return best_puzzle
    
def genetic_algorithm(puzzle):
    population_count = 1000
    max_generation = 300
    population = generate_population(puzzle, population_count)
    solution = main(puzzle, population, max_generation, population_count)
    return solution

if (ALGORITHM == "ga"):
    PUZZLE = read_puzzle(PUZZLE_PATH)
    PUZZLE_NUMBER = get_puzzle_number(PUZZLE_PATH)
    
    solved = genetic_algorithm(PUZZLE)
    if solved:
        write_puzzle(PUZZLE, GROUP_ID, ALGORITHM, PUZZLE_TYPE, PUZZLE_NUMBER)
        if is_valid_sudoku(PUZZLE):
            print("Solution is valid")
        else:
            print("Solution is invalid")
    else:
        print("No solution found by Genetic Algorithm.")

Puzzle written to Group36_ga_NA_Easy-P2.txt
Solution is invalid
